1. Load the packages

In [ ]:
import torch
import torchvision
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data import random_split
from torch.utils.data import Dataset,DataLoader,TensorDataset
from torch.autograd.function import Function
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torchvision import datasets, transforms
import torch.optim.lr_scheduler as lr_scheduler
import torch.utils.data as data
from PIL import Image
import os

2. Load the dataset


You need to download [MNIST-M](https://drive.google.com/file/d/1wFWMZ1z_cRVB0UYrMXACevZIXV4KTCwC/view) [1] to run this code. Then: 


2.1. If you run the code on Colab, you will need to put MNIST-M in the corresponding folder of your [Google Drive](https://drive.google.com/drive/u/0/my-drive).


2.2. If you run the code locally, you will need to put MNIST-M in the corresponding folder of your device.



[1] Ganin, Y., Ustinova, E., Ajakan, H., Germain, P., Larochelle, H., Laviolette, F., ... & Lempitsky, V. (2016). Domain-adversarial training of neural networks. The journal of machine learning research, 17(1), 2096-2030.

In [ ]:
# Mount the Google drive, please ignore this cell if you run the code locally.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path="/content/drive/MyDrive/domain adaption dataset/mnist_m.tar.gz" #change into the path you put the dataset downloaded
unzip_path="/content/sample_data"  # change into the path you want to unzip the file downloaded. 

!tar -xzvf "/content/drive/MyDrive/domain adaption dataset/mnist_m.tar.gz" -C "/content/sample_data"

Streaming output truncated to the last 5000 lines.
mnist_m/mnist_m_test/00008464.png
mnist_m/mnist_m_test/00004544.png
mnist_m/mnist_m_test/00000258.png
mnist_m/mnist_m_test/00004074.png
mnist_m/mnist_m_test/00006977.png
mnist_m/mnist_m_test/00004200.png
mnist_m/mnist_m_test/00005651.png
mnist_m/mnist_m_test/00008563.png
mnist_m/mnist_m_test/00008032.png
mnist_m/mnist_m_test/00003311.png
mnist_m/mnist_m_test/00003541.png
mnist_m/mnist_m_test/00003759.png
mnist_m/mnist_m_test/00005223.png
mnist_m/mnist_m_test/00008205.png
mnist_m/mnist_m_test/00007685.png
mnist_m/mnist_m_test/00005618.png
mnist_m/mnist_m_test/00001756.png
mnist_m/mnist_m_test/00005528.png
mnist_m/mnist_m_test/00008606.png
mnist_m/mnist_m_test/00005159.png
mnist_m/mnist_m_test/00005128.png
mnist_m/mnist_m_test/00001825.png
mnist_m/mnist_m_test/00001524.png
mnist_m/mnist_m_test/00003892.png
mnist_m/mnist_m_test/00006887.png
mnist_m/mnist_m_test/00007919.png
mnist_m/mnist_m_test/00005374.png
mnist_m/mnist_m_test/00000191.p

3. Load MNIST-M dataset

In [ ]:
class GetLoader(data.Dataset):
    def __init__(self, data_root, data_list, transform=None):
        self.root = data_root
        self.transform = transform

        f = open(data_list, 'r')
        data_list = f.readlines()
        f.close()

        self.n_data = len(data_list)

        self.img_paths = []
        self.img_labels = []

        for data in data_list:
            self.img_paths.append(data[:-3])
            self.img_labels.append(int(data[-2]))

    def __getitem__(self, item):
        img_paths, labels = self.img_paths[item], self.img_labels[item]
        imgs = Image.open(os.path.join(self.root, img_paths)).convert('RGB')

        if self.transform is not None:
            imgs = self.transform(imgs)
            labels = int(labels)

        return imgs, labels

    def __len__(self):
        return self.n_data

In [ ]:
image_size=32
img_transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    # transforms.Grayscale(num_output_channels=1)
])


In [ ]:
target_image_root="/content/sample_data/mnist_m/"

train_list = os.path.join(target_image_root, 'mnist_m_train_labels.txt')
test_list= os.path.join(target_image_root, 'mnist_m_test_labels.txt')


m_train = GetLoader(
    data_root=os.path.join(target_image_root, 'mnist_m_train'),
    data_list=train_list,
    transform=img_transform
)



m_test= GetLoader(
    data_root=os.path.join(target_image_root, 'mnist_m_test'),
    data_list=test_list,
    transform=img_transform
)


In [ ]:
print(len(m_train))
print(len(m_test))

59001
9001


4. Load MNIST dataset

In [ ]:
transform_mnist=  transforms.Compose([
                  transforms.Scale(image_size),
                  transforms.ToTensor(),
                  transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.size(0)==1 else x),
                  transforms.Normalize((0.5,), (0.5,))])
path='.\data'
mnist_train = datasets.MNIST(root=path, train=True,download=True, transform=transform_mnist)
mnist_test = datasets.MNIST(root=path, train=False,download=True, transform=transform_mnist)


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:317: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting .\data/MNIST/raw/train-images-idx3-ubyte.gz to .\data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting .\data/MNIST/raw/train-labels-idx1-ubyte.gz to .\data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting .\data/MNIST/raw/t10k-images-idx3-ubyte.gz to .\data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting .\data/MNIST/raw/t10k-labels-idx1-ubyte.gz to .\data/MNIST/raw



5. Center loss layers

In [ ]:
class CenterLoss(nn.Module):
    def __init__(self, num_classes, feat_dim, size_average=True):
        super(CenterLoss, self).__init__()
        self.centers = nn.Parameter(torch.randn(num_classes, feat_dim))
        self.centerlossfunc = CenterlossFunc.apply
        self.feat_dim = feat_dim
        self.size_average = size_average

    def forward(self, label, feat):
        batch_size = feat.size(0)
        feat = feat.view(batch_size, -1)
        # To check the dim of centers and features
        if feat.size(1) != self.feat_dim:
            raise ValueError("Center's dim: {0} should be equal to input feature's \
                            dim: {1}".format(self.feat_dim,feat.size(1)))
        batch_size_tensor = feat.new_empty(1).fill_(batch_size if self.size_average else 1)
        loss = self.centerlossfunc(feat, label, self.centers, batch_size_tensor)
        return loss


class CenterlossFunc(Function):
    @staticmethod
    def forward(ctx, feature, label, centers, batch_size):
        ctx.save_for_backward(feature, label, centers, batch_size)
        centers_batch = centers.index_select(0, label.long())
        return (feature - centers_batch).pow(2).sum() / 2.0 / batch_size

    @staticmethod
    def backward(ctx, grad_output):
        feature, label, centers, batch_size = ctx.saved_tensors
        centers_batch = centers.index_select(0, label.long())
        diff = centers_batch - feature
        # init every iteration
        counts = centers.new_ones(centers.size(0))
        ones = centers.new_ones(label.size(0))
        grad_centers = centers.new_zeros(centers.size())

        counts = counts.scatter_add_(0, label.long(), ones)
        grad_centers.scatter_add_(0, label.unsqueeze(1).expand(feature.size()).long(), diff)
        grad_centers = grad_centers/counts.view(-1, 1)
        return - grad_output * diff / batch_size, None, grad_centers / batch_size, None

#Testing function
def main(test_cuda=False):
    print('-'*80)
    device = torch.device("cuda" if test_cuda else "cpu")
    ct = CenterLoss(10,2,size_average=True).to(device)
    y = torch.Tensor([0,0,2,1]).to(device)
    feat = torch.zeros(4,2).to(device).requires_grad_()
    print (list(ct.parameters()))
    
    print (ct.centers.grad)
    out = ct(y,feat)
    print(out.item())
    out.backward()


if __name__ == '__main__':
    torch.manual_seed(999)
    main(test_cuda=False)
    if torch.cuda.is_available():
        main(test_cuda=True)

--------------------------------------------------------------------------------
[Parameter containing:
tensor([[-0.2528,  1.4072],
        [ 0.2910,  1.0365],
        [ 0.6396, -1.0857],
        [-1.6153,  1.5635],
        [ 0.1878, -0.9564],
        [-0.2440, -0.4153],
        [ 0.3259, -1.6059],
        [-0.5272,  0.3401],
        [-1.6526,  0.2108],
        [-1.3302,  0.4676]], requires_grad=True)]
None
0.8543729186058044
--------------------------------------------------------------------------------
[Parameter containing:
tensor([[ 0.9147, -1.1896],
        [-0.7501, -1.5465],
        [-0.4303,  0.3703],
        [ 0.4588,  0.2950],
        [ 0.2126, -0.1098],
        [-0.1349,  2.0038],
        [ 1.8339,  2.3544],
        [ 0.3990, -0.3584],
        [-0.2116,  0.5276],
        [ 0.5683, -0.6786]], device='cuda:0', requires_grad=True)]
None
0.9725638628005981


6. Define LetNet++

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1_1 = nn.Conv2d(3, 32, kernel_size=5, padding=2)
        self.prelu1_1 = nn.PReLU()
        self.conv1_2 = nn.Conv2d(32, 32, kernel_size=5, padding=2)
        self.prelu1_2 = nn.PReLU()
        self.conv2_1 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.prelu2_1 = nn.PReLU()
        self.conv2_2 = nn.Conv2d(64, 64, kernel_size=5, padding=2)
        self.prelu2_2 = nn.PReLU()
        self.conv3_1 = nn.Conv2d(64, 128, kernel_size=5, padding=2)
        self.prelu3_1 = nn.PReLU()
        self.conv3_2 = nn.Conv2d(128, 128, kernel_size=5, padding=2)
        self.prelu3_2 = nn.PReLU()
        self.preluip1 = nn.PReLU()
        self.ip1 = nn.Linear(128*16, 256)
        self.ip2 = nn.Linear(256, 10, bias=True)


    def forward(self, x):
        x = self.prelu1_1(self.conv1_1(x))
        x = self.prelu1_2(self.conv1_2(x))
        x = F.max_pool2d(x,2)
        x = self.prelu2_1(self.conv2_1(x))
        x = self.prelu2_2(self.conv2_2(x))
        x = F.max_pool2d(x,2)
        x = self.prelu3_1(self.conv3_1(x))
        x = self.prelu3_2(self.conv3_2(x))
        x = F.max_pool2d(x,2)
        x = x.view(-1, 128*16)

        ip1 = self.preluip1(self.ip1(x))
        ip2 = self.ip2(ip1)
        return ip1, F.log_softmax(ip2, dim=1)


In [ ]:
from torchsummary import summary
device = torch.device("cuda")
net = Net().to(device)

summary(net, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]           2,432
             PReLU-2           [-1, 32, 32, 32]               1
            Conv2d-3           [-1, 32, 32, 32]          25,632
             PReLU-4           [-1, 32, 32, 32]               1
            Conv2d-5           [-1, 64, 16, 16]          51,264
             PReLU-6           [-1, 64, 16, 16]               1
            Conv2d-7           [-1, 64, 16, 16]         102,464
             PReLU-8           [-1, 64, 16, 16]               1
            Conv2d-9            [-1, 128, 8, 8]         204,928
            PReLU-10            [-1, 128, 8, 8]               1
           Conv2d-11            [-1, 128, 8, 8]         409,728
            PReLU-12            [-1, 128, 8, 8]               1
           Linear-13                  [-1, 256]         524,544
            PReLU-14                  [

7. Define the training in one epoch

In [ ]:
def train(epoch,loss_weight,train_loader,model,nllloss,centerloss,optimizer4nn,optimzer4center,test_loader):
  ip1_loader = []
  idx_loader = []
  center_total=0
  nll_losss=0
  #Training in each epoch#
  for data, target in train_loader:
    data, target = data.to(device), target.to(device)

    ip1, pred = model(data)
    loss = nllloss(pred, target) + loss_weight * centerloss(target, ip1)
    
    optimizer4nn.zero_grad()
    optimzer4center.zero_grad()

    loss.backward()

    optimizer4nn.step()
    optimzer4center.step()
    ip1_loader.append(ip1)
    idx_loader.append((target))

    center_total=center_total+centerloss(target, ip1).item()
    nll_losss= nll_losss+nllloss(pred, target).item()*64
  model.eval()
  correct = 0
  for inputs, labels in test_loader:
    inputs, labels= inputs.cuda(),labels.cuda()
    ip1, outputs = model(inputs)
    preds = torch.max(outputs, 1)[1]
    correct += torch.sum(preds == labels.data)
  epoch_acc = correct.double() / len(test_loader.dataset)
  if epoch>45:
    print("Training... Epoch = %d" % epoch)
    print('Softmax:', nll_losss/len(train_loader.dataset))
    print('Center_loss:', center_total/len(train_loader))
    print('Testing acc:',epoch_acc.item())
  return model,epoch_acc

8. Define function for randomly selecting samples from source and target domains

In [ ]:


def get_index(labels,select_num):
  select_list=np.array([])
  remaining_list=np.array([])
  for i in range(10):
    full_list=np.where(labels==i)[0]
    ran_select=np.random.choice(full_list,select_num,replace=False)
    select_list=np.hstack((select_list,ran_select))

    remain_list=np.setdiff1d(full_list,ran_select)
    remaining_list=np.hstack((remaining_list,remain_list))

  return select_list.astype(int),remaining_list.astype(int)
def decompose_dataset(datesett):
  
  x_set=[]
  y=[]
  for i in range(len(datesett)):
    x,tem_y=datesett[i]
    x_set.append(x.numpy())
    y.append(tem_y)

  return np.array(x_set),np.array(y)

In [ ]:
print(len(mnist_train))
print(len(mnist_test))
print(len(m_train))
print(len(m_test))

60000
10000
59001
9001


9. Define the training function

In [ ]:
def trainss(types,weightss,center_step):


  src_label = np.array(mnist_train.targets)
  tar_label = np.array(m_train.img_labels)

  srctrain_index,_=get_index(src_label,5000)
  source_trainset=torch.utils.data.Subset(mnist_train, srctrain_index)
  source_trainset=source_trainset
  tartrain_index,tartest_index=get_index(tar_label,10)
  target_trainset=torch.utils.data.Subset(m_train, tartrain_index)

  train_set=torch.utils.data.ConcatDataset([source_trainset, target_trainset])


 

  batch_size=64
  if types=="sourceonly":
    train_loader= DataLoader(source_trainset, batch_size=batch_size, shuffle=True, num_workers=2)
    print(len(source_trainset))

  else:
    train_loader= DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
    print(len(train_set))
  


  test_set = m_test
  test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)
  print(len(test_set))


  # Model
  model = Net().to(device)


  # Softmax
  nllloss = nn.CrossEntropyLoss().to(device) #CrossEntropyLoss = log_softmax + NLLLoss

  loss_weight =  weightss

  # CenterLoss
  centerloss = CenterLoss(10, 256).to(device)

  # optimzer4nn
  optimizer4nn = optim.Adam(model.parameters(),lr=0.00025)

  sheduler = lr_scheduler.StepLR(optimizer4nn,20,gamma=0.8)

  # optimzer4center
  optimzer4center = torch.optim.SGD(centerloss.parameters(), lr =center_step)
  best=[0]
  eps=0
  print("Type:",types)
  for epoch in range(50):
    
    # print optimizer4nn.param_groups[0]['lr']
    model,accuracy=train(epoch+1, loss_weight,train_loader,model,nllloss,centerloss,optimizer4nn,optimzer4center,test_loader)
    sheduler.step()
    if accuracy > max(best):
      best.append(accuracy)
      eps=epoch

  print("Epoch:",str(eps),"Best acc:", str(max(best)))
  return model


10. Example

Examples below are baselines, Models 1, 2, and 3, on the MNIST to MNISTM experiment. Only one repetition for each is displyed.


Note that models will be trained from scratch. Training time should last for around **1 hours** for all models with one repetition if you use the GPU (GeForce RTX 3090).

In [ ]:
types="sourceonly"
lambda_para,center_step=0,0.5 
model=trainss(types,lambda_para,center_step)


types="softmax"
lambda_para,center_step=0,0.5 
model=trainss(types,lambda_para,center_step)

types="centerloss"
lambda_para,center_step=0.01,0.5
model=trainss(types,lambda_para,center_step)



50000
9001
Type: sourceonly
Training... Epoch = 46
Softmax: 3.1685806153802787e-09
Center_loss: 2544.076042780181
Testing acc: 0.6273747361404288
Training... Epoch = 47
Softmax: 2.0098675636859297e-09
Center_loss: 2657.997555471747
Testing acc: 0.6272636373736252
Training... Epoch = 48
Softmax: 1.3279910443486642e-09
Center_loss: 2769.451395098206
Testing acc: 0.6273747361404288
Training... Epoch = 49
Softmax: 8.463857720641954e-10
Center_loss: 2880.014660613311
Testing acc: 0.6269303410732141
Training... Epoch = 50
Softmax: 5.269049788125813e-10
Center_loss: 3002.146620182125
Testing acc: 0.6263748472391957
Epoch: 10 Best acc: tensor(0.6309, device='cuda:0', dtype=torch.float64)
50100
9001
Type: softmax
Training... Epoch = 46
Softmax: 1.9484927283263676e-07
Center_loss: 1265.1862419586375
Testing acc: 0.7154760582157539
Training... Epoch = 47
Softmax: 1.3251971343767668e-07
Center_loss: 1319.9721749063349
Testing acc: 0.7153649594489502
Training... Epoch = 48
Softmax: 8.88265534668989